# ESO Python Coffee - May 29, 2024
# (Short) Introdution to SHAP

### R. Carvajal (IA - FCUL, Portugal)

Shapley values (Shapley, 1953) are a method from coalitional game theory that tells us how to fairly distribute the dividends (the prediction in our case) among the features of a function (see a thorough description in its Wikipedia [article](https://en.wikipedia.org/wiki/Shapley_value)).

SHapley Additive exPlanations (SHAP, [documentation](https://shap.readthedocs.io/en/latest/) and [repository](https://github.com/shap/shap)) is a Python package that can obtain Shapley values (or a very good approximation) from general functions, including complex Machine Learning (ML) models.

These ML models are often treated as "black boxes" because their internal workings are difficult to understand. SHAP helps understanding them by revealing how each piece of data (a feature) influences the final prediction.

Contrary to other methods and packages, SHAP can extract information from individual predictions. This means we can go beyond understanding the overall importance of features and know how each prediction `moves' inside the function or model.

SHAP values (the output from SHAP) tell us how much each feature in a model (or other function) impacts its final predictions. They do this by comparing the actual model output to what the it would predict if that specific feature had a different value. Essentially, they calculate the difference between the prediction with a specific feature and the model's expected prediction without it.

In simpler terms, SHAP values show how much each feature "pushes" the model's prediction in a certain direction (positive or negative) compared to a baseline.  The higher the absolute SHAP value (positive or negative), the bigger the impact that feature has on the final prediction.

We will show two short examples of how to use and interpret SHAP values from simple functions.

First, we install and import all the packages we will need,

In [ ]:
# !pip install astropy
# !pip install shap

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from astropy.table import Table
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from PIL import Image
from joblib import dump, load
import shap
import helper_functions as helpers  # own file with functions

### Function and data set-up

For our first example, we will use a polynomial and we will try to determine their strongest factors.

We will create a polynomial of degree 5 of the form:

$$ f(x) = 1.5 + x_{1} + 2 x_{2}^{2} - 3 x_{3}^{3} + 0.8 x_{4}^{4} - 0.5 x_{5}^{5}$$

with

$$x = (x_{0}, x_{1}, x_{2}, x_{3}, x_{4}, x_{5})$$

In [ ]:
coeffs = [1.5, 1, 2, -3, 0.8, -0.5]
degree = 5
nelems = 10

We define that polynomial in Python

In [ ]:
def func_pol(x):
  # x is a 5D vector
  # 1.5 + x1 + 2 x2^2 -3 x3^3 + 0.8 x4^4 - 0.5 x5^5
  pol = 0
  for count in np.arange(degree + 1):
    pol += coeffs[count] * x[:, count]**(count)
  return pol

We create the set of input values for our function, x.

In this case, random values bewtween $-2$ and $2$.

Also, and for clarity, we name each column (feature) of the dataset.

In [ ]:
rng          = np.random.default_rng()
data_rand    = rng.uniform(low=-2, high=2, size=(nelems, degree + 1))
data_rand_df = pd.DataFrame(data_rand, columns=['Degree_0', 'Degree_1', 'Degree_2',
                                                'Degree_3', 'Degree_4', 'Degree_5'])
display(data_rand_df)

Now, we can check their output values from our polynomial

In [ ]:
func_pol(data_rand)

In order to understand the impact of each element of the polynomial, we can plot how it behaves when all the elements of the vectors are zero ($0$) except that of the analysed degree.

In [ ]:
helpers.plot_polynomial_degrees(func_pol, coeffs=coeffs, degree=degree)

We can see, for instance, that the first-degree term remains constant regardless of the value of $x_{0}$.

Then, the remaining terms behave as we would expect times their coefficient. In this way, the term with the largest range of values is the third-degree factor (i.e. $[-25,25]$, even more than the fift-degree term).


### SHAP analysis

Now, we start our SHAP analysis. First, we need to create an `Explainer` object with our function and the data it uses for training (in a ML context). Some relevant attributes are:

- `expected_value`
- `shap_values()`


Then, we can apply that explainer to the data we want to explain. It will output an `Explanation` object with the following relevant attributes:

- `feature_names`
- `data`
- `base_values`
- `values`

`expected_value` will output the mean value of all predictions for our dataset. This value is used by SHAP to obtain their impact on the final predictions. Recall that SHAP values are relative to their base value.

`feature_names` outputs the name of the features used in our traininig (or background) data.

`base_values` will, by default, replicate the `expected_value` from the `Explainer` for each feature. But, these `base_values` can be modified as needed.

There are several types of `Explainer` objects. A full list is shown [here](https://shap.readthedocs.io/en/latest/api.html).

We have selected the `KernelExplainer`, which implements a linear regression for the computation of feature importances of any model and function.

In [ ]:
explainer_func    = shap.KernelExplainer(func_pol, data_rand_df)
explanations_func = explainer_func(data_rand_df)
shap_values_func  = explanations_func.values

In [ ]:
explainer_func.expected_value

In [ ]:
explanations_func.feature_names

In [ ]:
explanations_func.base_values

From their theoretical definition, SHAP values are linear and can be combined and compared between them. For that reason, we are able to re-scale them to express the percentage of impact over the total prediction. We do this and also show the cumulative impact of all features.

In [ ]:
explanations_func_df = helpers.tabular_shap_vals(explanations_func)
display(explanations_func_df)

From the analysis of the previous table, we can see that the most impactful factor is the third-degree coefficient (much more than the fift-degree element). Also, if we created a new polynomial, but without the first and zeroth-degree coefficient, the final results would be almost the same of the original function ($\sim 95 \%$).

The SHAP package implements several graphical tools for the analysis of their values.
We will start with the simplest plot.

The bar plot shows the mean absolute value of the SHAP values among the target dataset. These values are sorted by importance as to highlight their relative importance.

In [ ]:
shap.plots.bar(shap_values=explanations_func,
                max_display=len(explanations_func.feature_names),
                show=True)

A second visualization is the Decision Plot. It shows how each data instance arrives to its final prediction or output after being fed to our target function or model.

The x-axis represents the model’s output. For a classification ML model, the units would be log odds. The plot is centered on the x-axis at `expected_value`. The y-axis lists the model’s features. By default, the features are ordered by descending importance. The importance is calculated over the observations plotted. Each observation’s prediction is represented by a colored line. At the top of the plot, each line strikes the x-axis at its corresponding observation’s predicted value. This value determines the color of the line on a spectrum. * Moving from the bottom of the plot to the top, SHAP values for each feature are added to the model’s base value. This shows how each feature contributes to the overall prediction. * At the bottom of the plot, the observations converge at explainer.expected_value.

We create a decision plot for all the values in our dataset.

In [ ]:
shap.decision_plot(explainer_func.expected_value,
                    shap_values_func,
                    data_rand_df,
                    plot_color='viridis')

We can see how relevant the third-degree term is for the final output. Also, we can notice the null impact of the zeroth-order factor.

It is also possible to create Decision Plots for each individual element of the dataset. We will do it for the three first element of our sample.

In each row, the plot shows the value of the selected element that goes into each coefficient. That is, the value of each feature.

In [ ]:
for count in np.arange(nelems)[:3]:
    shap.decision_plot(explainer_func.expected_value,
                        shap_values_func[count],
                        data_rand_df.loc[count],
                        plot_color='viridis')

The SHAP package can be used with regular functions, as in this example. It can also be applied to all sorts of ML models (random forests, image models, NNs, and more).

Additional plots are available to assess the impact of each feature in the final output of the studied functions and models.

## An Astronomical example: Redshift prediction in SDSS QSOs

Now, we will obtain SHAP values for a simple ML model that predicts redshift values for QSOs from the SDSS-DR16Q (Lyke et al. 2020) dataset with photometric measurements.

### Data set-up

We first load our data and select the relevant features: redshift, SDSS magnitudes, GALEX magnitudes, *WISE* magnitudes, 2MASS magnitudes, and *Gaia* proper motions.

In [ ]:
SDSS_data_table = Table.read('./data/DR16Q_v4_measured_test.fits')
SDSS_data_table.info

In [ ]:
cols_2_use_SDSS = ['Z', 'PSFMAG_u', 'PSFMAG_g', 'PSFMAG_r',
                    'PSFMAG_i', 'PSFMAG_z', 'FUV_MAG', 'NUV_MAG',
                    'W1_MAG', 'W2_MAG', 'JMAG', 'HMAG', 'KMAG',
                    'GAIA_PM_RA', 'GAIA_PM_DEC']
                    
SDSS_data_df = SDSS_data_table.to_pandas().loc[:, cols_2_use_SDSS]

Given the logarithmic nature of redshifts, we create a function for the comparison of true and predicted redshifts through their bias.

In [ ]:
def bias_z(y_true, y_pred, **kwargs):
    num = np.abs(y_true - y_pred)
    den = 1 + y_true
    return np.nanmedian(num / den)

bias_z_scorer = make_scorer(score_func=bias_z, greater_is_better=False)

We reserve a $20\%$ of the dataset for testing the model's performance.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(SDSS_data_df.drop(columns=['Z'], inplace=False),
                                                    SDSS_data_df.loc[:, 'Z'],
                                                    test_size=0.20,
                                                    random_state=42)
print(np.shape(X_train))
print(np.shape(X_test))

### Model creation, fitting, and scoring

We will use a Random Forest regressor for the prediction of redshifts. We create the object and train it.

In [ ]:
regr_SDSS = RandomForestRegressor(max_depth=4, random_state=42, verbose=1)
regr_SDSS.fit(X_train, y_train)

Now, we want to know how well our predictions are in the test subset. We calculate, with cross-validation, the bias of the sample. 

In [ ]:
test_scores_SDSS = -cross_val_score(regr_SDSS,
                                    X_test,
                                    y_test,
                                    cv=10,
                                    scoring=bias_z_scorer,
                                    verbose=0)

y_pred_test      = regr_SDSS.predict(X_test)

print('-'*30)
print(f'Testing bias is {np.nanmean(test_scores_SDSS):.3f} ± {np.nanstd(test_scores_SDSS):.3f}')
print('-'*30)

We can also plot the predicted and true values to see the performance of the model on the test set.
The model could do better, but this is not our goal in this notebook.

In [ ]:
helpers.plot_redshifts(y_test, y_pred_test)

### SHAP analysis

We create all SHAP objects and values.

In [ ]:
explainer_SDSS   = shap.TreeExplainer(regr_SDSS)
explanation_SDSS = explainer_SDSS(X_test)
shap_values_SDSS = explanation_SDSS.values

SHAP values can be re-scaled to be shown as percentages.

In [ ]:
explanations_SDSS_df = helpers.tabular_shap_vals(explanation_SDSS)
display(explanations_SDSS_df)

Bar plot for mean absolute SHAP values of test subset.

In [ ]:
shap.plots.bar(shap_values=explanation_SDSS,
                max_display=len(explanation_SDSS.feature_names),
                show=True)

We see that the most important feature, by far, is the *W1* magnitude (from *WiSE*), followed closely by the combination of GALEX magnitudes.

Also, *Gaia* proper motions make almost no difference in the prediction of QSO redshifts. Was this expected?

Discarding the last six features would not affect the final output by much.

Now, we display all test sources in a Decision plot. We can see that most of the impact starts to be noticeable from the u band (from bottom to top).

In [ ]:
shap.decision_plot(explainer_SDSS.expected_value,
                    shap_values_SDSS,
                    X_test,
                    plot_color='viridis')

For clarity, we plot the first ten sources of the test subset. We can see that the source with the highest predicted redshift has a behaviour different from the rest. Its r and u magnitudes are different, making it an outlier in this sample. Is it worth investigating it further?

In [ ]:
shap.decision_plot(explainer_SDSS.expected_value,
                    shap_values_SDSS[:10],
                    X_test.iloc[:10],
                    plot_color='viridis')

## Another astronomical (toy) example: image analysis

We want to use SHAP for the analysis of images. In principle, SHAP can be used for classifiers that take images as input.

In our case, we will use a function that 'predicts' if the mean value from the pixels in an image is larger than the median. In other words, we want to know, for no particular reason, if the distribution of pixel values is positively skewed or not.

![alt text](Skewness_definition.png "Title")

We first load a nice image of NGC3432. From Hubble, we can see very impressive star-forming regions throughout the galaxy.

For our model, we turn the image into an array.

In [ ]:
img = Image.open('./data/NGC3432.jpg')
img_galaxy = np.array(img)

In [ ]:
img

In [ ]:
display(img_galaxy)

In [ ]:
np.shape(img_galaxy)

We test our model with NGC3432. We can see that its skewness is positive.

In [ ]:
helpers.predict_galaxy(img_galaxy)

### SHAP analysis

Now, we want to create a SHAP explainer that takes an image and masks some regions to understand their impact in the final output. The larger the number of iterations, `n_evals`, the finer will be the result.

In [ ]:
batch_size = 60
n_evals = 1000 # 1000, 500, 300 evaluations

In [ ]:
# define a masker that is used to mask out partitions of the input image.
masker_shap = shap.maskers.Image('inpaint_telea', img_galaxy.shape)

# create an explainer with model and image masker
explainer = shap.Explainer(helpers.predict_galaxy, masker_shap)

The next block can take quite a while to run. I have run it previously and the results are saved in the repository. Thus, we can safely skip this step. But you can experiment with different number of iterations if you want.

In [ ]:
# feed only one image
# here we explain two images using 1000 evaluations of the underlying model to estimate the SHAP values
run_SHAP_block = False
if run_SHAP_block:
    shap_values_galaxy = explainer(
        np.array([img_galaxy]),
        max_evals=n_evals,
        batch_size=batch_size
    )

In [ ]:
if run_SHAP_block:
    try:
        dump(shap_values, 'shap_values/shap_galaxy.joblib', compress=True)
        print('SHAP values saved successfully!')
    except Exception as e:
        print('An error occurred while saving the values:', e)

We load the saved SHAP values.

In [ ]:
if not run_SHAP_block:
    try:
        shap_values_galaxy = load('shap_values/shap_galaxy.joblib')
        print('SHAP values loaded successfully!')
    except Exception as e:
        print('An error occurred while loading the values:', e)

Just to be sure, we check that our SHAP values have the same dimensions as the original image.

In [ ]:
(shap_values_galaxy.data.shape, shap_values_galaxy.values.shape)

Finally, we plot the SHAP values for NGC3432. We can see the impact that different regions of the image have in the final prediction. For instance, the central region of the galaxy makes it get negatively skewed.

In [ ]:
shap.image_plot(
    shap_values=shap_values_galaxy.values,
    pixel_values=shap_values_galaxy.data,
    labels=shap_values_galaxy.output_names,
)

Of course, SHAP values should be primarily applied in Machine Learning models, but we have see that it is possible to use them in very different contexts.

In conclusion, we can go beyond predictions with the use of feature importance methods. In particular, SHAP can help us obtaining very relevant information about the behaviour of the predicted elements inside models and functions. In Astrophysical contexts, we can use SHAP values to extract (hidden?) connections between features that can drive new discoveries.